# Imports

In [1]:
import logging
import os

from dotenv import load_dotenv

from core.enums import EnvironmentKey
from cmg.evaluators import CommitMessageGenerator
from cmg import evaluator
from core import (
    few_shot_high_level_cmg_chain,
    low_level_cmg_chain,
    zero_shot_high_level_cmg_chain
)
from core.models import CommitDataModel
from datapreparation import context_generator, example_generator

# Initialization

In [2]:
COMMIT_DATA_JSON_FILE_PATH = os.path.join("data", "cmg", "commits.json")
EXAMPLE_DATA_JSON_FILE_PATH = os.path.join("data", "cmg", "examples.json")
CONTEXT_DATA_PATH = os.path.join("data", "context")

DEFAULT_CONTEXT_GENERATION_OUTPUT_PATH = os.path.join(
    "data", "context"
)
DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.path.join(
    "out", "result", "highlevelcontext"
)
DEFAULT_CMG_OUTPUT_PATH = os.path.join("out", "result", "cmg")
DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH = os.path.join(
    "out", "result", "diffclassification"
)
DEFAULT_EXAMPLE_GENERATION_OUTPUT_PATH = os.path.join("out", "result", "example")

GENERATORS = [
    CommitMessageGenerator(
        "Zero-Shot High-Level Generator", zero_shot_high_level_cmg_chain
    ),
    CommitMessageGenerator(
        "Few-Shot High-Level Generator", few_shot_high_level_cmg_chain
    ),
    CommitMessageGenerator("Low-Level Generator", low_level_cmg_chain),
]

load_dotenv(dotenv_path=".env", verbose=True, override=True)
logging.basicConfig(level=logging.DEBUG)

In [3]:
CONTEXT_GENERATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.CONTEXT_GENERATION_OUTPUT_PATH.value,
        DEFAULT_CONTEXT_GENERATION_OUTPUT_PATH,
    )

HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.getenv(
        EnvironmentKey.HIGH_LEVEL_CONTEXT_OUTPUT_PATH.value,
        DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH,
    )

CMG_OUTPUT_PATH = os.getenv(
        EnvironmentKey.CMG_OUTPUT_PATH.value, DEFAULT_CMG_OUTPUT_PATH
    )

DIFF_CLASSIFICATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.DIFF_CLASSIFICATION_OUTPUT_PATH.value,
        DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH,
    )

EXAMPLE_GENERATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.EXAMPLE_GENERATION_OUTPUT_PATH.value,
        DEFAULT_EXAMPLE_GENERATION_OUTPUT_PATH,
    )

In [4]:
def get_commits(path: str) -> list[CommitDataModel]:
        with open(path, "r", encoding="utf-8") as file:
            json_string = file.read()

        return CommitDataModel.from_json(json_string)

COMMITS = get_commits(COMMIT_DATA_JSON_FILE_PATH)
EXAMPLE_COMMITS = get_commits(EXAMPLE_DATA_JSON_FILE_PATH)

# Generate Context

In [5]:
context_generator.generate_context(COMMITS + EXAMPLE_COMMITS, CONTEXT_GENERATION_OUTPUT_PATH)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): issues.apache.org:443
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/serverInfo HTTP/11" 200 None
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/issue/KAFKA-18520 HTTP/11" 200 2504


# Generate Examples

In [6]:
example_generator.generate_examples(COMMITS, CONTEXT_DATA_PATH, EXAMPLE_GENERATION_OUTPUT_PATH)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Given a Git diff and the relevant source code, write a concise summary of the code changes in a way that a non-technical person can understand. The query text must summarize the code changes in two very brief sentences.\n\nGit diff:\ndiff --git a/clients/src/main/java/org/apache/kafka/common/security/JaasUtils.java b/clients/src/main/java/org/apache/kafka/common/security/JaasUtils.java\nindex 742319c4f4..800283e56e 100644\n--- a/clients/src/main/java/org/apache/kafka/common/security/JaasUtils.java\n+++ b/clients/src/main/java/org/apache/kafka/common/security/JaasUtils.java\n@@ -16,67 +16,12 @@\n  */\n package org.apache.kafka.common.security;\n \n-import org.apache.kafka.common.KafkaException;\n-\n-import org.slf4j.Logger;\n-import org.slf4j.LoggerFactory;\n-\n-import javax.security.auth.login.Configuration;\n-\n public final class JaasUtils {

# Generate Commit Message

In [5]:
evaluator.evaluate(GENERATORS, COMMITS, CMG_OUTPUT_PATH)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Classify the Git diff into one of the following six software maintenance activities: feat, fix, perf, test, refactor, or chore. Return the activity that best matches the code changes. Refer to the definitions below for each activity.\n\nfeat: introducing new features into the system.\nfix: fixing existing bugs or issues in the system.\nperf: improving the performance of the system.\ntest: adding, modifying, or deleting test cases.\nrefactor: changes made to the internal structure of software to make it easier to understand and cheaper to modify without changing its observable behavior, including code styling.\nchore: regular maintenance tasks, such as updating dependencies or build tasks.\n\nAvoid adding any additional comments or annotations to the cla

DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 202 33
